## Introduction

This tutorial basically follows official documentation https://www.tensorflow.org/get_started/get_started.

In [15]:
import tensorflow as tf

## Constant

Constant cannot be printed using standard print() function.

In [16]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

(<tf.Tensor 'Const_4:0' shape=() dtype=float32>, <tf.Tensor 'Const_5:0' shape=() dtype=float32>)


## Session
In order to evalute nodes, we must run the computational graph with session.

In [17]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


### Operation as Node

In [18]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3)", sess.run(node3))

('node3: ', <tf.Tensor 'Add_1:0' shape=() dtype=float32>)
('sess.run(node3)', 7.0)


## Placeholders
Placeholder accepts extenal inputs such as training data, actual value can be provided to placeholder later.

In [19]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + provides a shortcut for tf.add(a, b)
multiplier_node = a * b

In [20]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

print(sess.run(multiplier_node, {a: 3, b: 4.5}))
print(sess.run(multiplier_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]
13.5
[  2.  12.]


In [21]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


## Variable
Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [22]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)

linear_model = W * x + b

In [23]:
init = tf.global_variables_initializer()
sess.run(init)

In [24]:
print(sess.run(linear_model, {x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [25]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

print(sess.run(squared_deltas, {x: [1,2,3,4], y:[0,-1,-2,-3]}))
print(sess.run(loss, {x: [1,2,3,4], y:[0,-1,-2,-3]}))

[  0.           1.68999982   6.75999928  15.21000099]
23.66


In [26]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


## Training

In [27]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [28]:
sess.run(init)
print(sess.run([W, b]))
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
    if ((i+1)%100 == 0):
        print('Iteration %s' % i, sess.run([W, b]))
print(sess.run([W, b]))

[array([ 0.30000001], dtype=float32), array([-0.30000001], dtype=float32)]
('Iteration 99', [array([-0.84079814], dtype=float32), array([ 0.53192717], dtype=float32)])
('Iteration 199', [array([-0.95227844], dtype=float32), array([ 0.85969269], dtype=float32)])
('Iteration 299', [array([-0.98569524], dtype=float32), array([ 0.95794231], dtype=float32)])
('Iteration 399', [array([-0.99571204], dtype=float32), array([ 0.98739296], dtype=float32)])
('Iteration 499', [array([-0.99871475], dtype=float32), array([ 0.99622124], dtype=float32)])
('Iteration 599', [array([-0.99961478], dtype=float32), array([ 0.99886739], dtype=float32)])
('Iteration 699', [array([-0.99988455], dtype=float32), array([ 0.99966055], dtype=float32)])
('Iteration 799', [array([-0.99996537], dtype=float32), array([ 0.99989825], dtype=float32)])
('Iteration 899', [array([-0.99998957], dtype=float32), array([ 0.99996936], dtype=float32)])
('Iteration 999', [array([-0.9999969], dtype=float32), array([ 0.99999082], dtyp

## Complete Program

In [29]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1,2,3,5]
y_train = [-0.987,1.298,2.576,6.889]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s, b: %s, loss: %s" % (curr_W, curr_b, curr_loss))

W: [ 1.93119788], b: [-2.86679292], loss: 0.227108


## tf.contrib.learn

In [30]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=input_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bbf00c710>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:S

In [31]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    # ModelFnOps connects subgraphs we built to the
    # appropriate functionality.
    return tf.contrib.learn.ModelFnOps(
        mode=mode, predictions=y,
        loss=loss,
        train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bbd95a910>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpkxrOco/model.ckpt.
INFO:tensorflow:loss = 76.3532140042, step = 1
INFO:tensorflow:global_step/sec: 1057.02
INFO:tensorflow:loss = 0.0104099627451, step = 101 (0.096 sec)
INFO:tensorflow:global_step/sec: 1292.69
INFO:tensorflow:loss = 0.000162538926523, step = 201 (0.077 sec)
INFO:tensorflow: